In [6]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import glob
import ipywidgets as widgets
from inspect import signature

# Time series

In [5]:
runs = glob.glob(
    "/Users/caspar/Developer/mscthesis/data/cases/bomex/output-gcc/run*/tmser.001.nc"
)
ensemble = xr.open_mfdataset(runs, combine="nested", concat_dim="run")

nvidia = xr.open_dataset(
    "/Users/caspar/Developer/mscthesis/data/cases/bomex/output-nvidia/tmser.001.nc"
)

In [86]:
variable_menu = widgets.Dropdown(
    options=list(nvidia.keys()),
    description="Variable"
)

ui = widgets.HBox([variable_menu])

def update(variable: str) -> None:
    mean = ensemble[variable].mean(dim="run")
    var = ensemble[variable].var(dim="run")
    
    fix, ax = plt.subplots()
    mean.plot.line(x="time", ax=ax)
    ax.fill_between(
        x=var["time"], y1=(2 * np.sqrt(var)) + mean, y2=(-2 * np.sqrt(var)) + mean
    )
    nvidia[variable].plot(x="time", ax=ax)
    
out = widgets.interactive_output(update, {'variable': variable_menu})

display(ui, out)

Output()

# Profiles

In [9]:
runs = glob.glob(
    "/Users/caspar/Developer/mscthesis/data/cases/bomex/output-gcc/run*/profiles.001.nc"
)
ensemble = xr.open_mfdataset(runs, combine="nested", concat_dim="run")

nvidia = xr.open_dataset(
    "/Users/caspar/Developer/mscthesis/data/cases/bomex/output-nvidia/profiles.001.nc"
)

In [11]:
variable_menu = widgets.Dropdown(
    options=list(nvidia.keys()),
    description="Variable"
)

timestamps_menu = widgets.IntSlider(
    value=0,
    min=0,
    max=len(ensemble.time)-1,
    step=1,
    description="Timestamp",
    continuous_update=False,
    orientation="horizontal"
)

ui = widgets.HBox([variable_menu, timestamps_menu])

def update(variable: str, time: int) -> None:
    mean = ensemble[variable].isel(time=time).mean(dim="run")
    var = ensemble[variable].isel(time=time).var(dim="run")
    
    fix, ax = plt.subplots()
    try:
        mean.plot.line(y="zt", ax=ax, label="GCC ensemble")
        ax.fill_betweenx(
            y=var["zt"], x1=(2 * np.sqrt(var)) + mean, x2=(-2 * np.sqrt(var)) + mean
        )
        nvidia[variable].isel(time=time).plot.line(y="zt", ax=ax, label="OpenACC")
    except:
        mean.plot.line(y="zm", ax=ax, label="GCC ensemble")
        ax.fill_betweenx(
            y=var["zm"], x1=(2 * np.sqrt(var)) + mean, x2=(-2 * np.sqrt(var)) + mean
        )
        nvidia[variable].isel(time=time).plot.line(y="zm", ax=ax, label="OpenACC")
    ax.legend()

out = widgets.interactive_output(update, {'variable': variable_menu, 'time': timestamps_menu})

display(ui, out)

Output()

In [77]:
nvidia['cfrac'].isel(time=-1)

<xarray.DataArray 'cfrac' (zt: 80)>
array([0.      , 0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.000122, 0.005615, 0.025146, 0.037354, 0.038208, 0.032959, 0.026978,
       0.02356 , 0.02124 , 0.019165, 0.0177  , 0.017212, 0.014893, 0.014526,
       0.011719, 0.011108, 0.009155, 0.00769 , 0.006348, 0.004272, 0.003418,
       0.001831, 0.000977, 0.000122, 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
       0.      , 0.      , 0.      ], dtype=float32)
Coordinates:
  * zt       (zt) float32 20.0 60.0 100.0 140.0 ... 3.1e+03 3.14e+03 3.18e+03
    time     float32 3.6e+04
Attributes:
    long_name:  Cloud fraction
    units:      -